In [1]:
import imageio.v2 as iio
import tensorflow as tf
import tensorflow.keras as ks
from transformers import ViTImageProcessor, TFViTForImageClassification

2024-07-11 12:56:03.005412: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-11 12:56:03.359976: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-11 12:56:03.360044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-11 12:56:03.368019: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-11 12:56:03.396883: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-11 12:56:03.398858: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# Create an instance of ViTImageProcessor and use ViT base model as pretrained model
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
# Create an instance of ViTForImageClassification and use ViT base model as pretrained model
model = TFViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
model.summary(expand_nested=True)

All PyTorch model weights were used when initializing TFViTForImageClassification.

All the weights of TFViTForImageClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


Model: "tf_vi_t_for_image_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit (TFViTMainLayer)        multiple                  85798656  
                                                                 
 classifier (Dense)          multiple                  769000    
                                                                 
Total params: 86567656 (330.23 MB)
Trainable params: 86567656 (330.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
def load_image_data(path):
    '''Function will load image data and process it with tensorflow decode_image function.
    Parameters:
    path <str>: file path string to load the image from

    Return:
    <tf.Tensor>: Decoded image with shape (height, width, channels)
    '''
    image = tf.io.read_file(path)
    return tf.io.decode_image(image)

@tf.py_function(Tout=tf.float32)
def apply_feature_extractor(x):
    '''Function will apply "feature_extractor" to image data. Decorator allows usage of this function inside tf.data.Dataset.map().
    Parameters:
    x <tf.Tensor>: image data

    Return:
    <tf.Tensor>: features of image data
    '''
    return feature_extractor(images=x, return_tensors='tf')['pixel_values']

# Create tf.data.Dataset containing all ".jpg" file names in folder
image_pipeline = tf.data.Dataset.list_files('*.jpg')
# Load the image date from file names
image_pipeline = image_pipeline.map(lambda x: load_image_data(x))
# Batch Dataset
image_pipeline = image_pipeline.batch(1)
# Apply feature_extractor to all image data
image_pipeline = image_pipeline.map(lambda x: apply_feature_extractor(x))

In [34]:
for i, val in enumerate(image_pipeline):
    print(val)

tf.Tensor(
[[[[-0.8745098  -0.8901961  -0.92156863 ... -0.4980392  -0.4980392
    -0.4980392 ]
   [-0.8509804  -0.8901961  -0.90588236 ... -0.4980392  -0.4980392
    -0.4980392 ]
   [-0.90588236 -0.9529412  -0.9529412  ... -0.4980392  -0.4980392
    -0.4980392 ]
   ...
   [-0.372549   -0.38039213 -0.3960784  ... -0.4980392  -0.5058824
    -0.5137255 ]
   [-0.40392154 -0.41176468 -0.41960782 ... -0.5137255  -0.5294118
    -0.5372549 ]
   [-0.41960782 -0.41960782 -0.42745095 ... -0.5058824  -0.52156866
    -0.5372549 ]]

  [[-0.96862745 -0.9843137  -1.         ... -0.47450978 -0.47450978
    -0.47450978]
   [-0.9529412  -0.9764706  -0.9843137  ... -0.47450978 -0.47450978
    -0.47450978]
   [-0.99215686 -1.         -1.         ... -0.47450978 -0.47450978
    -0.47450978]
   ...
   [-0.41176468 -0.41960782 -0.4352941  ... -0.5372549  -0.5529412
    -0.5529412 ]
   [-0.4588235  -0.46666664 -0.47450978 ... -0.56078434 -0.5764706
    -0.58431375]
   [-0.47450978 -0.47450978 -0.4823529  ... -

In [35]:
model.predict(image_pipeline)

OperatorNotAllowedInGraphError: in user code:

    File "/home/felbus/transformers/lib/python3.10/site-packages/tf_keras/src/engine/training.py", line 2436, in predict_function  *
        return step_function(self, iterator)
    File "/home/felbus/transformers/lib/python3.10/site-packages/tf_keras/src/engine/training.py", line 2409, in run_step  *
        outputs = model.predict_step(data)
    File "/home/felbus/transformers/lib/python3.10/site-packages/tf_keras/src/engine/training.py", line 2377, in predict_step  *
        return self(x, training=False)
    File "/home/felbus/transformers/lib/python3.10/site-packages/tf_keras/src/engine/training.py", line 558, in error_handler  *
        return fn(*args, **kwargs)
    File "/home/felbus/transformers/lib/python3.10/site-packages/tf_keras/src/engine/training.py", line 588, in __call__  *
        return super().__call__(*args, **kwargs)
    File "/home/felbus/transformers/lib/python3.10/site-packages/tf_keras/src/engine/training.py", lin

In [28]:
pic = tf.io.read_file('sample.jpg')
pic = tf.io.decode_image(pic)
pic_features = feature_extractor(pic, return_tensors='tf')
prediction = model.predict(pic_features['pixel_values'])
pic_features['pixel_values']

1/1 [==============================] - 0s 431ms/step


<tf.Tensor: shape=(1, 3, 224, 224), dtype=float32, numpy=
array([[[[ 0.11372554,  0.12156868,  0.15294123, ..., -0.4588235 ,
          -0.4588235 , -0.4588235 ],
         [ 0.06666672,  0.10588241,  0.17647064, ..., -0.41960782,
          -0.42745095, -0.42745095],
         [ 0.07450986,  0.12941182,  0.21568632, ..., -0.38039213,
          -0.38823527, -0.3960784 ],
         ...,
         [-0.6       , -0.5137255 , -0.38039213, ...,  0.23921573,
           0.12941182,  0.09803927],
         [-0.5137255 , -0.44313723, -0.3333333 , ...,  0.2941177 ,
           0.15294123,  0.05882359],
         [-0.46666664, -0.41960782, -0.34117645, ...,  0.27843142,
           0.1686275 ,  0.05882359]],

        [[-0.04313725, -0.03529412, -0.01960784, ..., -0.45098037,
          -0.45098037, -0.45098037],
         [-0.09019607, -0.04313725,  0.00392163, ..., -0.41176468,
          -0.41960782, -0.41960782],
         [-0.0745098 , -0.01960784,  0.04313731, ..., -0.372549  ,
          -0.38039213, -0.3

In [70]:
category_tensor = tf.math.argmax(prediction.logits, axis=1)


AttributeError: 'tuple' object has no attribute 'rank'